In [7]:
from data_loader import get_data
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data loader


In [8]:
signals, snr, labels_id, label_dict = get_data()

{1: 'N-PSK8',
 0: 'N-QAM16',
 2: 'N-QPSK',
 4: 'W-PSK8-V1',
 5: 'W-PSK8-V2',
 3: 'W-QAM16'}